In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from torchtext.utils import download_from_url, extract_archive

import nltk
import json
from tqdm import tqdm
import numpy as np

import time
import os
import random
from collections import Counter
from collections import defaultdict

In [2]:
torch.manual_seed(1)
random.seed(1)
np.random.seed(1)

In [3]:
def download_and_extract(url, root):
    os.makedirs(root, exist_ok=True)
    dataset_tar = download_from_url(snli_url, root=root)
    _ = extract_archive(dataset_tar)

In [4]:
snli_url = "http://nlp.stanford.edu/projects/snli/snli_1.0.zip"
root = ".data"

download_and_extract(snli_url, root)

In [5]:
class NLIDataset(Dataset):
    
    def __init__(self, path, field_tokenizers, field_numericalizers=dict()):
        
        data_iter = self._load_data(path)
        self.data = self._process_data(data_iter, field_tokenizers, field_numericalizers)
    
    def _load_data(self, path):
        with open(path, 'r') as f:
            for line in f:
                example = json.loads(line)
                if example['gold_label'] == '-':
                    continue
                yield example
                
    def _process_data(self, data_iter, field_tokenizers, field_numericalizers):
        data = []
        for example in tqdm(data_iter):
            _example = dict()
            for field, tokenizer in field_tokenizers.items():
                numericalizer = field_numericalizers.get(field)
                if tokenizer is not None:
                    _example[field] = tokenizer(example[field])
                else:
                    _example[field] = example[field]
                if numericalizer is not None:
                    _example[field] = numericalizer(_example[field])
            data.append(_example)
        return data        
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [6]:
class Tokenizer:
    
    def __init__(self, lower=True, max_length=250):
        
        self.lower = lower
        self.max_length = max_length
        
    def tokenize(self, text):
        
        assert isinstance(text, str)
        
        tokens = nltk.word_tokenize(text)
        
        if self.max_length is not None:
            tokens = tokens[:self.max_length]
            
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        return tokens

In [7]:
sentence_tokenizer = Tokenizer(lower=True, max_length=250)

In [8]:
field_tokenizers = {'gold_label': None,
                    'sentence1': sentence_tokenizer.tokenize,
                    'sentence2': sentence_tokenizer.tokenize}

train_data = NLIDataset('.data/snli_1.0/snli_1.0_train.jsonl', 
                        field_tokenizers)

549367it [01:22, 6654.79it/s]


In [9]:
train_data[0]

{'gold_label': 'neutral',
 'sentence1': ['a',
  'person',
  'on',
  'a',
  'horse',
  'jumps',
  'over',
  'a',
  'broken',
  'down',
  'airplane',
  '.'],
 'sentence2': ['a',
  'person',
  'is',
  'training',
  'his',
  'horse',
  'for',
  'a',
  'competition',
  '.']}

In [10]:
def build_token_freqs(data):
    freqs = defaultdict(Counter)
    for example in data:
        for field_name, field_value in example.items():
            if isinstance(field_value, list):
                freqs[field_name].update(field_value)
            else:
                freqs[field_name].update([field_value])
    return freqs 

In [11]:
freqs = build_token_freqs(train_data)

In [12]:
freqs.keys()

dict_keys(['gold_label', 'sentence1', 'sentence2'])

In [13]:
freqs['gold_label'].most_common()

[('entailment', 183416), ('contradiction', 183187), ('neutral', 182764)]

In [14]:
class Vocab:
    def __init__(self, freqs, max_size=None, min_freq=1, unk_token='<unk>', 
                 pad_token='<pad>', special_tokens=[]):

        assert isinstance(freqs, Counter)

        self.max_size = max_size
        self.min_freq = min_freq
        self.unk_token = unk_token
        self.pad_token = pad_token
        self.special_tokens = special_tokens

        self.stoi, self.itos = self.create_vocab(freqs)

    def create_vocab(self, freqs):

        stoi = dict()
        itos = list()

        if self.unk_token is not None:
            itos.append(self.unk_token)
        if self.pad_token is not None:
            itos.append(self.pad_token)
        for token in self.special_tokens:
            itos.append(token)

        for token, count in freqs.most_common(self.max_size):
            if token in itos:
                continue
            if count < self.min_freq:
                break
            else:
                itos.append(token)

        stoi.update({t: i for i, t in enumerate(itos)})

        return stoi, itos
    
    def numericalize(self, tokens):
        
        if isinstance(tokens, str):
            return self.stoi.get(tokens, self.stoi.get(self.unk_token))
            
        return [self.stoi.get(t, self.stoi.get(self.unk_token)) for t in tokens]
    
    def denumericalize(self, indexes):
        
        if isinstance(indexes, int):
            return self.itos[indexes]
        
        return [self.itos[i] for i in indexes]
    
    def __getitem__(self, token):
        return self.stoi.get(token, self.stoi.get(self.unk_token))
    
    def __len__(self):
        return len(self.itos)

In [15]:
sentence_freqs = freqs['sentence1'] + freqs['sentence2']
label_freqs = freqs['gold_label']

In [16]:
sentence_freqs.most_common(20)

[('a', 1438921),
 ('.', 962726),
 ('the', 534667),
 ('in', 407046),
 ('is', 373534),
 ('man', 266099),
 ('on', 235836),
 ('and', 206204),
 ('are', 199110),
 ('of', 192082),
 ('with', 169233),
 ('woman', 137620),
 ('two', 122071),
 ('people', 121150),
 (',', 114347),
 ('to', 113880),
 ('at', 98656),
 ('wearing', 81006),
 ('an', 80206),
 ('his', 72442)]

In [17]:
sentence_vocab = Vocab(sentence_freqs, min_freq=2)
label_vocab = Vocab(label_freqs, unk_token=None, pad_token=None)

In [18]:
print(f'Unique tokens in sentence vocabulary: {len(sentence_vocab)}')
print(f'Unique tokens in label vocabulary: {len(label_vocab)}')

Unique tokens in sentence vocabulary: 25547
Unique tokens in label vocabulary: 3


In [19]:
label_vocab.stoi

{'entailment': 0, 'contradiction': 1, 'neutral': 2}

In [20]:
field_tokenizers = {'gold_label': None,
                    'sentence1': sentence_tokenizer.tokenize,
                    'sentence2': sentence_tokenizer.tokenize}

field_numericalizers = {'gold_label': label_vocab.numericalize,
                        'sentence1': sentence_vocab.numericalize,
                        'sentence2': sentence_vocab.numericalize}

train_data = NLIDataset('.data/snli_1.0/snli_1.0_train.jsonl', 
                        field_tokenizers,
                        field_numericalizers)

549367it [01:25, 6396.28it/s]


In [21]:
train_data[0]

{'gold_label': 2,
 'sentence1': [2, 43, 8, 2, 194, 204, 78, 2, 1150, 39, 799, 3],
 'sentence2': [2, 43, 6, 1154, 21, 194, 38, 2, 441, 3]}

In [22]:
def denumericalize_example(example, field_denumericalizers):
    _example = dict()
    for field, denumericalizer in field_denumericalizers.items():
        _example[field] = denumericalizer(example[field])
    return _example

In [23]:
field_denumericalizers = {'gold_label': label_vocab.denumericalize,
                          'sentence1': sentence_vocab.denumericalize,
                          'sentence2': sentence_vocab.denumericalize}

In [24]:
denumericalize_example(train_data[0], field_denumericalizers)

{'gold_label': 'neutral',
 'sentence1': ['a',
  'person',
  'on',
  'a',
  'horse',
  'jumps',
  'over',
  'a',
  'broken',
  'down',
  'airplane',
  '.'],
 'sentence2': ['a',
  'person',
  'is',
  'training',
  'his',
  'horse',
  'for',
  'a',
  'competition',
  '.']}

In [25]:
valid_data = NLIDataset('.data/snli_1.0/snli_1.0_dev.jsonl', 
                        field_tokenizers,
                        field_numericalizers)

test_data = NLIDataset('.data/snli_1.0/snli_1.0_test.jsonl', 
                        field_tokenizers,
                        field_numericalizers)

9842it [00:01, 6227.64it/s]
9824it [00:01, 6082.69it/s]


In [26]:
class SNLICollator:
    def __init__(self, field_pad_idxs, device, batch_first=False):
        
        self.field_pad_idxs = field_pad_idxs
        self.device = device
        self.batch_first = batch_first
        
    def collate(self, batch):
        
        labels = []
        premises = []
        hypotheses = []
        
        for example in batch:
            labels.append(example['gold_label'])
            premises.append(torch.LongTensor(example['sentence1']))
            hypotheses.append(torch.LongTensor(example['sentence2']))
            
        premises = pad_sequence(premises, 
                                padding_value=self.field_pad_idxs['sentence1'],
                                batch_first=self.batch_first).to(self.device)
        hypotheses = pad_sequence(hypotheses,
                                  padding_value=self.field_pad_idxs['sentence2'],
                                  batch_first=self.batch_first).to(self.device)
        labels = torch.LongTensor(labels).to(self.device)
        
        return labels, premises, hypotheses

In [27]:
field_pad_idxs = {'gold_label': label_vocab[label_vocab.pad_token],
                  'sentence1': sentence_vocab[sentence_vocab.pad_token],
                  'sentence2': sentence_vocab[sentence_vocab.pad_token]}

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [29]:
collator = SNLICollator(field_pad_idxs, device)

In [30]:
batch_size = 128

train_iterator = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collator.collate)
valid_iterator = DataLoader(valid_data, batch_size=batch_size, shuffle=False, collate_fn=collator.collate)
test_iterator = DataLoader(test_data, batch_size=batch_size, shuffle=False, collate_fn=collator.collate)

In [31]:
class NLIBiLSTM(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim,
                 hidden_dim,
                 n_lstm_layers,
                 bidirectional,
                 n_fc_layers,
                 output_dim, 
                 dropout, 
                 pad_idx):
        
        super().__init__()
                                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.translation = nn.Linear(embedding_dim, hidden_dim)
        
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=n_lstm_layers, 
                            bidirectional=bidirectional, 
                            dropout=dropout if n_lstm_layers > 1 else 0)
        
        fc_dim = hidden_dim * 2 if bidirectional else hidden_dim
        
        fcs = [nn.Linear(fc_dim * 2, fc_dim * 2) for _ in range(n_fc_layers)]
        
        self.fcs = nn.ModuleList(fcs)
        
        self.fc_out = nn.Linear(fc_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, prem, hypo):

        prem_seq_len, batch_size = prem.shape
        hypo_seq_len, _ = hypo.shape
        
        #prem = [prem sent len, batch size]
        #hypo = [hypo sent len, batch size]
        
        embedded_prem = self.embedding(prem)
        embedded_hypo = self.embedding(hypo)
        
        #embedded_prem = [prem sent len, batch size, embedding dim]
        #embedded_hypo = [hypo sent len, batch size, embedding dim]
        
        translated_prem = F.relu(self.translation(embedded_prem))
        translated_hypo = F.relu(self.translation(embedded_hypo))
        
        #translated_prem = [prem sent len, batch size, hidden dim]
        #translated_hypo = [hypo sent len, batch size, hidden dim]
        
        outputs_prem, (hidden_prem, cell_prem) = self.lstm(translated_prem)
        outputs_hypo, (hidden_hypo, cell_hypo) = self.lstm(translated_hypo)

        #outputs_x = [sent len, batch size, n directions * hid dim]
        #hidden_x = [n layers * n directions, batch size, hid dim]
        #cell_x = [n layers * n directions, batch size, hid dim]
        
        if self.lstm.bidirectional:
            hidden_prem = torch.cat((hidden_prem[-1], hidden_prem[-2]), dim=-1)
            hidden_hypo = torch.cat((hidden_hypo[-1], hidden_hypo[-2]), dim=-1)
        else:
            hidden_prem = hidden_prem[-1]
            hidden_hypo = hidden_hypo[-1]
        
        #hidden_x = [batch size, fc dim]

        hidden = torch.cat((hidden_prem, hidden_hypo), dim=1)

        #hidden = [batch size, fc dim * 2]
            
        for fc in self.fcs:
            hidden = fc(hidden)
            hidden = F.relu(hidden)
            hidden = self.dropout(hidden)
        
        prediction = self.fc_out(hidden)
        
        #prediction = [batch size, output dim]
        
        return prediction

In [32]:
input_dim = len(sentence_vocab)
embedding_dim = 100
hidden_dim = 300
n_lstm_layers = 1
bidirectional = True
n_fc_layers = 3
output_dim = len(label_vocab)
dropout = 0.2
pad_idx = sentence_vocab[sentence_vocab.pad_token]

model = NLIBiLSTM(input_dim,
                  embedding_dim,
                  hidden_dim,
                  n_lstm_layers,
                  bidirectional,
                  n_fc_layers,
                  output_dim,
                  dropout,
                  pad_idx)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,357,003 trainable parameters


In [34]:
def get_pretrained_glove_vectors(vocab, dimension, root='.data'):
    url = 'http://nlp.stanford.edu/data/glove.6B.zip'
    download_and_extract(url, root)
    vectors = torch.zeros(len(vocab), dimension)
    found = 0
    with open(f'{root}/glove.6B.{dimension}d.txt', 'r') as f:
        for line in tqdm(f):
            token, *vector = line.split()
            if token in vocab.stoi:
                found += 1
                vector = [float(v) for v in vector]
                vectors[vocab.stoi[token]] = torch.FloatTensor(vector)
    print(f'{found/len(vocab)*100:.2f}% of vocab had a pre-trained embedding!')
    return vectors

In [35]:
pretrained_vectors = get_pretrained_glove_vectors(sentence_vocab, embedding_dim)

400000it [00:02, 175982.21it/s]

86.45% of vocab had a pre-trained embedding!


In [36]:
model.embedding.weight.data.copy_(pretrained_vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2709,  0.0440, -0.0203,  ..., -0.4923,  0.6369,  0.2364],
        ...,
        [-0.0924, -0.4158, -0.1312,  ..., -0.0713, -0.7363, -0.1793],
        [ 0.2884, -0.8653, -0.3934,  ...,  0.3048,  1.3187, -0.5727],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [37]:
model.embedding.weight.data[sentence_vocab[sentence_vocab.pad_token]] = torch.zeros(embedding_dim)

In [38]:
optimizer = optim.Adam(model.parameters())

In [39]:
criterion = nn.CrossEntropyLoss()

In [40]:
model = model.to(device)
criterion = criterion.to(device)

In [41]:
def categorical_accuracy(preds, labels):
    max_preds = preds.argmax(dim=-1) # get the index of the max probability
    correct = max_preds.eq(labels)
    return correct.sum() / torch.FloatTensor([labels.shape[0]])

In [42]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, prems, hypos in iterator:
                
        optimizer.zero_grad()
        
        #prems = [prems sent len, batch size]
        #hypos = [hypos sent len, batch size]
        
        predictions = model(prems, hypos)
        
        #predictions = [batch size, output dim]
        #labels = [batch size]
        
        loss = criterion(predictions, labels)
                
        acc = categorical_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, prems, hypos in iterator:
                        
            predictions = model(prems, hypos)
            
            loss = criterion(predictions, labels)
                
            acc = categorical_accuracy(predictions, labels)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [44]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
n_epochs = 10

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 20s
	Train Loss: 0.725 | Train Acc: 68.59%
	 Val. Loss: 0.609 |  Val. Acc: 74.91%
Epoch: 02 | Epoch Time: 1m 19s
	Train Loss: 0.580 | Train Acc: 76.61%
	 Val. Loss: 0.540 |  Val. Acc: 78.57%
Epoch: 03 | Epoch Time: 1m 20s
	Train Loss: 0.516 | Train Acc: 79.64%
	 Val. Loss: 0.511 |  Val. Acc: 79.77%
Epoch: 04 | Epoch Time: 1m 20s
	Train Loss: 0.471 | Train Acc: 81.65%
	 Val. Loss: 0.499 |  Val. Acc: 80.25%
Epoch: 05 | Epoch Time: 1m 18s
	Train Loss: 0.433 | Train Acc: 83.32%
	 Val. Loss: 0.494 |  Val. Acc: 80.79%
Epoch: 06 | Epoch Time: 1m 18s
	Train Loss: 0.399 | Train Acc: 84.78%
	 Val. Loss: 0.506 |  Val. Acc: 81.04%
Epoch: 07 | Epoch Time: 1m 18s
	Train Loss: 0.366 | Train Acc: 86.14%
	 Val. Loss: 0.507 |  Val. Acc: 80.90%
Epoch: 08 | Epoch Time: 1m 18s
	Train Loss: 0.336 | Train Acc: 87.39%
	 Val. Loss: 0.551 |  Val. Acc: 80.46%
Epoch: 09 | Epoch Time: 1m 18s
	Train Loss: 0.307 | Train Acc: 88.57%
	 Val. Loss: 0.562 |  Val. Acc: 80.50%
Epoch: 10 | Epoch T

In [46]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.507 |  Test Acc: 80.67%


In [47]:
def infer(model, premise, hypothesis, sentence_tokenizer, sentence_numericalizer, label_vocab, device):
    
    model.eval()
    
    if isinstance(premise, str):
        premise = sentence_tokenizer(premise)
        
    if isinstance(hypothesis, str):
        hypothesis = sentence_tokenizer(hypothesis)
        
    premise = sentence_numericalizer(premise)
    hypothesis = sentence_numericalizer(hypothesis)
        
    premise = torch.LongTensor(premise).unsqueeze(1).to(device)
    hypothesis = torch.LongTensor(hypothesis).unsqueeze(1).to(device)
            
    prediction = model(premise, hypothesis)
        
    prediction = prediction.argmax(dim=-1).item()
    
    return label_vocab.itos[prediction]

In [48]:
premise = 'a blue bird in a tree.'
hypothesis = 'an azure bird on a branch.'

infer(model, premise, hypothesis, sentence_tokenizer.tokenize, sentence_vocab.numericalize, label_vocab, device)

'entailment'

In [49]:
premise = 'a blue bird in a tree.'
hypothesis = 'a blue bird, flying in the sky.'

infer(model, premise, hypothesis, sentence_tokenizer.tokenize, sentence_vocab.numericalize, label_vocab, device)

'contradiction'

In [59]:
premise = 'a blue bird in a tree.'
hypothesis = 'a bird, just sitting there.'

infer(model, premise, hypothesis, sentence_tokenizer.tokenize, sentence_vocab.numericalize, label_vocab, device)

'neutral'